In [1]:
%pylab inline
%matplotlib inline

# Global Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import os
import sys
import pickle
from pprint import pprint
from time import time
import datetime
from time import gmtime, strftime
import statsmodels.api as sm
from patsy import dmatrices

# Scikit-Learn imports
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import train_test_split

from sklearn.cross_validation import cross_val_score
from sklearn import metrics

# Local Imports
path = str(os.path.expanduser('~')) + '/git/predictEPL/config'
sys.path.append(path)
import paths

sys.path.append(paths.UTILS)
import useful_methods

Populating the interactive namespace from numpy and matplotlib


In [2]:
def ReadHashDf(RESULT_FILE_NAME):
    # Read game_infos as df
    dfGameInfo = useful_methods.csv_dic_df(paths.READ_PATH_GAME_INFO + 'game_infos.csv')
    dfGameInfo = useful_methods.DropNanGames(dfGameInfo).copy().reset_index(drop=True)
    dfGameInfo.GW = [int(gw) for gw in dfGameInfo.GW]
    dfGameInfo = dfGameInfo.sort_values(['GW', 'away_team'], ascending=[True, True]).copy().reset_index(drop=True)

    # Read Hash Emolex Model result
    df = useful_methods.csv_dic_df(paths.READ_PATH_RESULTS + RESULT_FILE_NAME)
    df.GW = [int(gw) for gw in df.GW]
    df = df.sort_values(['GW', 'away_team'], ascending=[True, True]).copy().reset_index(drop=True)


    # Combine 2 dfs
    df['score_ht_away'] = [int(item) for item in dfGameInfo.score_ht_away]
    df['score_ht_home'] = [int(item) for item in dfGameInfo.score_ht_home]
    df['score_ft_away'] = [int(item) for item in dfGameInfo.score_ft_away]
    df['score_ft_home'] = [int(item) for item in dfGameInfo.score_ft_home]

    df['pn_away_neg'] = [float(item) for item in df.pn_away_neg]
    df['pn_away_pos'] = [float(item) for item in df.pn_away_pos]
    df['pn_home_neg'] = [float(item) for item in df.pn_home_neg]
    df['pn_home_pos'] = [float(item) for item in df.pn_home_pos]

    # 'home_win': 1, 'away_win': 0, 'draw': 2
    def Labeling(goal_diff):
        if goal_diff > 0:
            return 1
        elif goal_diff < 0:
            return 0
        else:
            return 2

    df['goal_diff_ht'] = df.score_ht_home - df.score_ht_away
    df['goal_diff_ft'] = df.score_ft_home - df.score_ft_away
    df['result'] = [Labeling(item) for item in df.goal_diff_ft]


    # **********************************************
    # Add previous 4 games points sum
    dfBe4GameSum = useful_methods.csv_dic_df(paths.DATA_HOME + "EPL/info_before_4game_sum.csv")

    team_homes = list(df.home_team)
    team_aways = list(df.away_team)
    GWs = [int(gw) for gw in df.GW]

    be_4game_sum_home = []
    be_4game_sum_away = []

    for index in range(len(GWs)):
        team_home = team_homes[index]
        team_away = team_aways[index]
        gw = GWs[index]

        be_4game_sum_home.append(int(dfBe4GameSum[dfBe4GameSum.team == team_home][str(gw)]))
        be_4game_sum_away.append(int(dfBe4GameSum[dfBe4GameSum.team == team_away][str(gw)]))

    df['be_4game_sum_away'] = be_4game_sum_away
    df['be_4game_sum_home'] = be_4game_sum_home    

    return df

In [3]:
# Create df for models.
def CreateDfForModel(RESULT_FILE_NAME, ht_draw=False, ft_wld=False):
    df = ReadHashDf(RESULT_FILE_NAME)
        
    # only for Win or Lose
    if not ft_wld:
        df = df[df.result != 2].copy().reset_index(drop=True)
    
    # HT: Equal
    if ht_draw:
        df = df[df.goal_diff_ht == 0].copy().reset_index(drop=True)
    
    dta = pd.DataFrame()
    
    # Teams
    dta['team_home'] = df.home_team
    dta['team_away'] = df.away_team

    # POS, NEG scores by percentage %
    dta['pn_home_pos'] = df.pn_home_pos / (df.pn_home_pos + df.pn_home_neg)
    dta['pn_home_neg'] = df.pn_home_neg / (df.pn_home_pos + df.pn_home_neg)
    dta['pn_away_pos'] = df.pn_away_pos / (df.pn_away_pos + df.pn_away_neg)
    dta['pn_away_neg'] = df.pn_away_neg / (df.pn_away_pos + df.pn_away_neg)
    
    dta['pn_diff_pos'] = dta['pn_home_pos'] - dta['pn_away_pos']
    
    # HF scores
    dta['score_ht_home'] = df.score_ht_home
    dta['score_ht_away'] = df.score_ht_away
    
    dta['goal_diff_ht'] = df['goal_diff_ht']

    # 'home_win': 1, 'away_win': 0, 'draw': 2
    dta['result'] = df.result
    
    # be_4game_sum
    dta['be_4game_sum_home'] = df.be_4game_sum_home / 12.0
    dta['be_4game_sum_away'] = df.be_4game_sum_away / 12.0
    
    return dta

In [4]:
def EplPnTable(RESULT_FILE_NAME):
    df = CreateDfForModel(RESULT_FILE_NAME, ft_wld=True)
    df = df.drop(df.columns[-6:], axis = 1)

    teams = list(set(df.team_home))
    teams.sort()

    pn_home_pos_avg = []
    pn_home_neg_avg = []
    pn_away_pos_avg = []
    pn_away_neg_avg = []

    for team in teams:
        df_home = df[(df.team_home == team)]
        df_away = df[(df.team_away == team)]

        pn_home_pos_avg.append(df_home.pn_home_pos.mean())
        pn_home_neg_avg.append(df_home.pn_home_neg.mean())

        pn_away_pos_avg.append(df_away.pn_away_pos.mean())
        pn_away_neg_avg.append(df_away.pn_away_neg.mean())


    dfSentTable = pd.DataFrame()
    dfSentTable['team'] = teams

    dfSentTable['pn_home_pos_avg'] = pn_home_pos_avg
    dfSentTable['pn_home_neg_avg'] = pn_home_neg_avg

    dfSentTable['pn_away_pos_avg'] = pn_away_pos_avg
    dfSentTable['pn_away_neg_avg'] = pn_away_neg_avg

    dfSentTable['pos'] = (dfSentTable.pn_home_pos_avg + dfSentTable.pn_away_pos_avg) / 2
    dfSentTable['neg'] = (dfSentTable.pn_home_neg_avg + dfSentTable.pn_away_neg_avg) / 2

    return dfSentTable.sort_values(['pos'], ascending=[False]).reset_index(drop=True)

## EPL Sentiment(pos, neg) Table

In [7]:
# Definings
RESULT_FILE_NAME_HT = "hash_all_ht.csv"
RESULT_FILE_NAME_FT = "hash_all_ft.csv"

dfPnTable = EplPnTable(RESULT_FILE_NAME_FT)
dfPnTable.sort_values(['pos'], ascending=[False]).reset_index(drop=True)

,team,pn_home_pos_avg,pn_home_neg_avg,pn_away_pos_avg,pn_away_neg_avg,pos,neg
0,Swansea,0.676706,0.323294,0.680572,0.319428,0.678639,0.321361
1,Leicester,0.684079,0.315921,0.657052,0.342948,0.670565,0.329435
2,Bournemouth,0.607644,0.392356,0.620010,0.379990,0.613827,0.386173
3,Tottenham,0.604307,0.395693,0.620519,0.379481,0.612413,0.387587
4,WestHam,0.615745,0.384255,0.573418,0.426582,0.594581,0.405419
5,Crystal,0.570834,0.429166,0.579106,0.420894,0.574970,0.425030
6,Southampton,0.603293,0.396707,0.535263,0.464737,0.569278,0.430722
7,Arsenal,0.589149,0.410851,0.527349,0.472651,0.558249,0.441751
8,City,0.551241,0.448759,0.509390,0.490610,0.530315,0.469685
9,Sunderland,0.523558,0.476442,0.476303,0.523697,0.499931,0.500069


## EPL Score Table
Exclude GW: [1, 2, 3, 4, 13]

In [8]:
dfScore = useful_methods.ReadEplScoreTable()

# drop weeks
dfScore = dfScore.drop(dfScore.columns[[1, 2, 3, 4, 5, 13]], axis=1)

# sum all scores
dfScore['pts'] = dfScore.sum(axis=1)

# sort by overall points
dfScore = dfScore.sort_values(['pts'], ascending=False).reset_index(drop=True)
dfScore[['team', 'pts']]

,team,pts
0,Tottenham,38
1,Leicester,36
2,Arsenal,36
3,City,31
4,WestHam,29
5,Watford,29
6,Stoke,28
7,United,27
8,Liverpool,27
9,Southampton,25
